<a href="https://colab.research.google.com/github/AyaElsawyElghaysh/AyaElsawyELghaysh/blob/main/AyaElsawy__GraphFrams_P_S__Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The following section is for Colab Users.
### Just run the following code cells

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

     |████████████████████████████████| 281.3 MB 56 kB/s 
     |████████████████████████████████| 199 kB 68.2 MB/s 
     |████████████████████████████████| 154 kB 60.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar -P /content/spark-3.0.2-bin-hadoop2.7/jars/
!cp /content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar /content/spark-3.0.2-bin-hadoop2.7/graphframes-0.8.2-spark3.0-s_2.12.zip

--2022-06-24 17:47:08--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 54.230.18.120, 54.230.18.27, 54.230.18.61, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|54.230.18.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247882 (242K) [binary/octet-stream]
Saving to: ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  1.49MB/s    in 0.2s    

2022-06-24 17:47:08 (1.49 MB/s) - ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’ saved [247882/247882]



In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [ ]:
import findspark
findspark.init()

In [ ]:
!export PYSPARK_SUBMIT_ARGS="--master local[*] pyspark-shell"
!export PYSPARK_DRIVER_PYTHON=jupyter
!export PYSPARK_DRIVER_PYTHON_OPTS=notebook

In [ ]:
from pyspark.sql import SparkSession
from graphframes import *

spark = SparkSession.builder.master("local[*]").appName("GraphFrames").getOrCreate()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

**************************************************************************
**************************************************************************
**************************************************************************

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read departuredelays.csv in Edge DataFrame
### Read airport-codes-na.txt in Vertix DataFrame (the separator is Tab i.e sep = '\t' )

#### The US flight delays data set has five columns:
- The <b>date</b> column contains an integer like 02190925 . When converted, this maps to 02-19 09:25 am.
- The <b>delay</b> column gives the delay in minutes between the scheduled and actual departure times. Early departures show negative numbers.
- The <b>distance</b> column gives the distance in miles from the origin airport to the destination airport.
- The <b>origin</b> column contains the origin IATA airport code.
- The <b>destination</b> column contains the destination IATA airport code.

#### The airport-codes data set has four columns:
- The <b>IATA</b> column contains IATA airport code.
- The <b>City, State, and Country</b> columns contains information about the airport location. 

In [ ]:
df_air = spark.read.option("header","true" )\
               .option("delimiter","\t")\
               .option("inferSchema", "true" )\
               .csv("/content/drive/MyDrive/Spark_data/data/airport-codes-na.txt")
df_air.show()
df_air.printSchema()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



In [ ]:
depart=spark.read.csv("/content/drive/MyDrive/Spark_data/data/departuredelays.csv",header=True,inferSchema=True)

In [ ]:
depart.show(truncate=False)

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|6    |602     |ABE   |ATL        |
|1020600|-8   |369     |ABE   |DTW        |
|1021245|-2   |602     |ABE   |ATL        |
|1020605|-4   |602     |ABE   |ATL        |
|1031245|-4   |602     |ABE   |ATL        |
|1030605|0    |602     |ABE   |ATL        |
|1041243|10   |602     |ABE   |ATL        |
|1040605|28   |602     |ABE   |ATL        |
|1051245|88   |602     |ABE   |ATL        |
|1050605|9    |602     |ABE   |ATL        |
|1061215|-6   |602     |ABE   |ATL        |
|1061725|69   |602     |ABE   |ATL        |
|1061230|0    |369     |ABE   |DTW        |
|1060625|-3   |602     |ABE   |ATL        |
|1070600|0    |369     |ABE   |DTW        |
|1071725|0    |602     |ABE   |ATL        |
|1071230|0    |369     |ABE   |DTW        |
|1070625|0    |602     |ABE   |ATL        |
|1071219|0    |569     |ABE   |ORD        |
|1080600|0    |369     |ABE   |D

### In the vertix DataFrame, drop any duplicated rows with the same  IATA code.

In [ ]:
vertix_1=df_air.distinct()
vertix_1.show()

+--------------+-------+-------+----+
|          City|  State|Country|IATA|
+--------------+-------+-------+----+
|    Clarksburg|     WV|    USA| CKB|
|    Fort Dodge|     IA|    USA| FOD|
|       Redmond|     OR|    USA| RDM|
|        Valdez|     AK|    USA| VDZ|
|       Lebanon|     NH|    USA| LEB|
|         Aspen|     CO|    USA| ASE|
| Rouyn-Noranda|     PQ| Canada| YUY|
|       Iqaluit|Nunavut| Canada| YFB|
|       Toronto|     ON| Canada| YYZ|
|    Dillingham|     AK|    USA| DLG|
|   Fort Nelson|     BC| Canada| YYE|
|      Pellston|     MI|    USA| PLN|
|     St. Cloud|     MN|    USA| STC|
|St. Petersburg|     FL|    USA| PIE|
|    Great Bend|     KS|    USA| GBD|
|          Hilo|     HI|    USA| Big|
|       Kelowna|     BC| Canada| YLW|
|       Bemidji|     MN|    USA| BJI|
|        London|     ON| Canada| YXU|
|      Longview|     TX|    USA| GGG|
+--------------+-------+-------+----+
only showing top 20 rows



### In the edges DataFrame:
- Rename the <b>date</b> columns to become <b>tripid</b>.
- Rename the <b>origin</b> columns to become <b>src</b>.
- Rename the <b>destination</b> columns to become <b>dst</b>.

In [ ]:
depart_2=depart.withColumnRenamed("date","tripid")
depart_3=depart_2.withColumnRenamed("origin","src")
depart_4=depart_3.withColumnRenamed("destination","dst")

In [ ]:
depart_4.printSchema()

root
 |-- tripid: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)



### In the Vertix DataFrame:
- Rename the <b>IATA</b> columns to become <b>id</b>.

In [ ]:
vertix_2=vertix_1.withColumnRenamed("IATA","id")

In [ ]:
vertix_2.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- id: string (nullable = true)



### Create GraphFrame from Vertix and Edges DataFrames

In [ ]:
gf=GraphFrame(vertix_2,depart_4)
gf.edges.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
|1030605|    0|     602|ABE|ATL|
|1041243|   10|     602|ABE|ATL|
|1040605|   28|     602|ABE|ATL|
|1051245|   88|     602|ABE|ATL|
|1050605|    9|     602|ABE|ATL|
|1061215|   -6|     602|ABE|ATL|
|1061725|   69|     602|ABE|ATL|
|1061230|    0|     369|ABE|DTW|
|1060625|   -3|     602|ABE|ATL|
|1070600|    0|     369|ABE|DTW|
|1071725|    0|     602|ABE|ATL|
|1071230|    0|     369|ABE|DTW|
|1070625|    0|     602|ABE|ATL|
|1071219|    0|     569|ABE|ORD|
|1080600|    0|     369|ABE|DTW|
+-------+-----+--------+---+---+
only showing top 20 rows



In [ ]:
gf.vertices.show()

+--------------+-------+-------+---+
|          City|  State|Country| id|
+--------------+-------+-------+---+
|    Clarksburg|     WV|    USA|CKB|
|    Fort Dodge|     IA|    USA|FOD|
|       Redmond|     OR|    USA|RDM|
|        Valdez|     AK|    USA|VDZ|
|       Lebanon|     NH|    USA|LEB|
|         Aspen|     CO|    USA|ASE|
| Rouyn-Noranda|     PQ| Canada|YUY|
|       Iqaluit|Nunavut| Canada|YFB|
|       Toronto|     ON| Canada|YYZ|
|    Dillingham|     AK|    USA|DLG|
|   Fort Nelson|     BC| Canada|YYE|
|      Pellston|     MI|    USA|PLN|
|     St. Cloud|     MN|    USA|STC|
|St. Petersburg|     FL|    USA|PIE|
|    Great Bend|     KS|    USA|GBD|
|          Hilo|     HI|    USA|Big|
|       Kelowna|     BC| Canada|YLW|
|       Bemidji|     MN|    USA|BJI|
|        London|     ON| Canada|YXU|
|      Longview|     TX|    USA|GGG|
+--------------+-------+-------+---+
only showing top 20 rows



### Determine the number of airports

In [ ]:
gf.vertices.describe().show()

+-------+----------+-----+-------+----+
|summary|      City|State|Country|  id|
+-------+----------+-----+-------+----+
|  count|       526|  523|    526| 526|
|   mean|      null| null|   null|null|
| stddev|      null| null|   null|null|
|    min|Abbotsford|   AB| Canada| ABE|
|    max|      Yuma|   YT|    USA| YZV|
+-------+----------+-----+-------+----+



In [ ]:
gf.vertices.select(count(col('id'))).show()

+---------+
|count(id)|
+---------+
|      526|
+---------+



### Determine the number of trips 

In [ ]:
gf.edges.describe().show()

+-------+-----------------+------------------+-----------------+-------+-------+
|summary|           tripid|             delay|         distance|    src|    dst|
+-------+-----------------+------------------+-----------------+-------+-------+
|  count|          1391578|           1391578|          1391578|1391578|1391578|
|   mean|2180446.584000322|12.079802928761449|690.5508264718184|   null|   null|
| stddev|838031.1536740534| 38.80773374985635|513.6628153663347|   null|   null|
|    min|          1010005|              -112|               21|    ABE|    ABE|
|    max|          3312359|              1642|             4330|    YUM|    YUM|
+-------+-----------------+------------------+-----------------+-------+-------+



In [ ]:
gf.edges.select(count(col("tripid"))).show()

+-------------+
|count(tripid)|
+-------------+
|      1391578|
+-------------+



### What is the longest delay?

In [ ]:
gf.edges.select(max(col("delay")).alias("MAX_DELAY")).show()

+---------+
|MAX_DELAY|
+---------+
|     1642|
+---------+



### Find out the number of delayed flights vs. early flights (flights that departed before actual time)

In [ ]:
gf.edges.filter(col("delay")<0).count()

668729

In [ ]:
gf.edges.filter(col("delay")>0).count()

591727

### What flight destinations departing SFO are most likely to have significant delays? Select the top 10
#### Hint: you should get the average delay for each destination for trips that depart from SFO only

### Find the Incoming connections to the airport sorted in Desc. order.

In [ ]:
gf.inDegrees.orderBy('inDegree', ascending=False).show()

+---+--------+
| id|inDegree|
+---+--------+
|ATL|   90434|
|DFW|   66050|
|ORD|   61967|
|LAX|   53601|
|DEN|   50921|
|IAH|   42700|
|PHX|   39721|
|SFO|   38988|
|LAS|   32994|
|CLT|   28388|
|MCO|   27959|
|EWR|   27652|
|LGA|   25469|
|BOS|   25360|
|SLC|   25323|
|JFK|   23484|
|DTW|   23310|
|SEA|   23074|
|MSP|   22385|
|MIA|   21805|
+---+--------+
only showing top 20 rows



### Find the Outgoing connections from the airport sorted in Desc. order.

In [ ]:
gf.outDegrees.orderBy('outDegree', ascending=False).show()

+---+---------+
| id|outDegree|
+---+---------+
|ATL|    91484|
|DFW|    68482|
|ORD|    64228|
|LAX|    54086|
|DEN|    53148|
|IAH|    43361|
|PHX|    40155|
|SFO|    39483|
|LAS|    33107|
|CLT|    28402|
|MCO|    28313|
|EWR|    27656|
|SLC|    25868|
|LGA|    25458|
|BOS|    25348|
|MSP|    24031|
|JFK|    23572|
|DTW|    23421|
|SEA|    23078|
|MIA|    21817|
+---+---------+
only showing top 20 rows



### Use motif finding to answer this question: which delays could we blame on SFO?
#### Hint: this practically means that SFO is a transit station

In [ ]:
gf.find("(v1)-[SFO]->(v2)").show()

+--------------------+--------------------+--------------------+
|                  v1|                 SFO|                  v2|
+--------------------+--------------------+--------------------+
|[Allentown, PA, U...|[1011245, 6, 602,...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1020600, -8, 369...|[Detroit, MI, USA...|
|[Allentown, PA, U...|[1021245, -2, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1020605, -4, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1031245, -4, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1030605, 0, 602,...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1041243, 10, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1040605, 28, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1051245, 88, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1050605, 9, 602,...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1061215, -6, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[1061725, 69, 602...|[Atlanta, GA, USA...|
|[Allentown, PA, U...|[10

### Determine Airport Ranking in Desc. order using PageRank algorithm

In [ ]:
rs=gf.pageRank(resetProbability=.01,maxIter=10)
flights=rs.vertices.orderBy("pagerank",ascending=False)
flights.show()

+--------------+-----+-------+---+------------------+
|          City|State|Country| id|          pagerank|
+--------------+-----+-------+---+------------------+
|       Atlanta|   GA|    USA|ATL| 33.96309070020015|
|        Dallas|   TX|    USA|DFW|24.824283143819542|
|       Chicago|   IL|    USA|ORD|23.380064781830225|
|   Los Angeles|   CA|    USA|LAX|20.028146392517304|
|        Denver|   CO|    USA|DEN|19.159892249109248|
|       Houston|   TX|    USA|IAH|15.945627536817861|
|       Phoenix|   AZ|    USA|PHX|14.877539591441929|
| San Francisco|   CA|    USA|SFO|14.633841499481823|
|     Las Vegas|   NV|    USA|LAS|12.371796694213915|
|     Charlotte|   NC|    USA|CLT|10.470743193006143|
|        Newark|   NJ|    USA|EWR|10.165802288086578|
|       Orlando|   FL|    USA|MCO|10.135934840422118|
|Salt Lake City|   UT|    USA|SLC| 9.679616724705348|
|      New York|   NY|    USA|LGA|  9.51032873001114|
|        Boston|   MA|    USA|BOS| 9.321359553228293|
|       Seattle|   WA|    US

## Determine the most popular flights (single city hops)

In [ ]:
flights.select("City").limit(10).show()

+-------------+
|         City|
+-------------+
|      Atlanta|
|       Dallas|
|      Chicago|
|  Los Angeles|
|       Denver|
|      Houston|
|      Phoenix|
|San Francisco|
|    Las Vegas|
|    Charlotte|
+-------------+



### Find and Save a Subragph that obtained from the following pattern:
#### The flight starts from an airport and return back to the same airport through 2 other airports.

In [ ]:
sub_graph=gf.find("(v1)-[e1]->(v2);(v2)-[e2]->(v3);(v3)-[e3]->(v1)")

In [ ]:
sub=sub_graph.select("v2","v3")

In [ ]:
sub.show()